# Microsoft Agent Framework - Learning Notebook

**Purpose:** This notebook is designed to explore all the capabilities of the Microsoft Agent Framework for learning and experimentation.

---

## What is Microsoft Agent Framework?

Microsoft Agent Framework is an **open-source development kit** for building AI agents and multi-agent workflows for .NET and Python. It brings together and extends ideas from **Semantic Kernel** and **AutoGen** projects, combining their strengths while adding new capabilities.

### Key Capabilities

| Category | Description |
|----------|-------------|
| **AI Agents** | Individual agents that use LLMs to process user inputs, call tools and MCP servers to perform actions, and generate responses |
| **Workflows** | Graph-based workflows that connect multiple agents and functions to perform complex, multi-step tasks |

### Building Blocks

The framework provides foundational components:
- **Model Clients** - Chat completions and responses (Azure OpenAI, OpenAI, Azure AI)
- **Agent Session** - State management for conversations
- **Context Providers** - Agent memory capabilities
- **Middleware** - Intercepting agent actions
- **MCP Clients** - Tool integration via Model Context Protocol

### When to Use AI Agents

AI agents excel at:
- 🎧 **Customer Support** - Multi-modal queries with tool lookups
- 📚 **Education & Tutoring** - Personalized learning with knowledge bases
- 💻 **Code Generation** - Implementation, reviews, and debugging
- 🔬 **Research Assistance** - Web search, document summarization

### When NOT to Use AI Agents

> *"If you can write a function to handle the task, do that instead of using an AI agent."*

Avoid agents for:
- Highly structured tasks with predefined rules
- Well-defined sequences of operations
- Tasks requiring more than ~20 tools (use workflows instead)

---

## Prerequisites

Before running this notebook:

1. ✅ **Azure subscription** with access to Azure OpenAI
2. ✅ **Azure OpenAI resource** with a deployed model (e.g., `gpt-4o-mini`)
3. ✅ **Azure CLI** installed and authenticated (`az login`)
4. ✅ **`.env` file** with your configuration (see README.md)

## Install Python Packages

To use Microsoft Agent Framework with Azure OpenAI, install the following Python packages:

```bash
pip install agent-framework --pre python-dotenv nest_asyncio
```

> **Note:** `nest_asyncio` is required for Python 3.10 compatibility to allow nested event loops in Jupyter notebooks.

In [7]:
%pip install agent-framework --pre python-dotenv nest_asyncio


[notice] A new release of pip is available: 25.1.1 -> 26.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Load Environment Variables

The `.env` file contains your Azure OpenAI configuration. The `python-dotenv` library loads these variables into the environment so the SDK can access them automatically.

Required variables:
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME` - Your deployed model name
- `AZURE_OPENAI_ENDPOINT` - Your Azure OpenAI endpoint URL
- `AZURE_OPENAI_API_KEY` - Your API key (optional if using Azure CLI auth)
- `API_VERSION` - The API version to use

In [8]:
import nest_asyncio
nest_asyncio.apply()

from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

print("Environment variables loaded successfully")

Environment variables loaded successfully


## Create the Agent

First, create a chat client for communicating with Azure OpenAI using the environment variables configured earlier.

Then, create the agent by providing instructions and a name for the agent.

In [9]:
import asyncio
from agent_framework.azure import AzureOpenAIChatClient
from azure.identity import AzureCliCredential

agent = AzureOpenAIChatClient(credential=AzureCliCredential()).as_agent(
    instructions="You are good at telling jokes.",
    name="Joker"
)

## Run the Agent

To run the agent, call the `run` method on the agent instance, providing the user input. The agent will return a response object, and accessing the `.text` property provides the text result from the agent.

In [10]:
async def run_agent():
    result = await agent.run("Tell me a joke about a pirate.")
    print(result.text)

asyncio.run(run_agent())

Why did the pirate go to school? 

Because he wanted to improve his "arrrticulation!"


```markdown
## Run the Agent with Streaming

To run the agent with streaming, call the `run_stream` method on the agent instance, providing the user input. The agent will stream a list of update objects, and accessing the `.text` property on each update object provides the part of the text result contained in that update.
```

In [11]:
# Stream the response token by token
async def stream_agent():
    async for update in agent.run_stream("Tell me a joke about a pirate."):
        if update.text:
            print(update.text, end="", flush=True)
    print()  # New line after streaming is complete

asyncio.run(stream_agent())

Why did the pirate go to school?

Because he wanted to improve his “arrrrrrrticulation!” 🏴‍☠️


## Multimodal Input with ChatMessage

You can pass `ChatMessage` objects with multiple content types, including text and images, to enable multimodal interactions with the agent. This is useful when you want the agent to analyze or respond to visual content along with textual instructions.

In [12]:
from agent_framework import ChatMessage, Content, Role

message = ChatMessage(
    role=Role.USER,
    contents=[
        Content.from_text("Tell me a joke about this image?"),
        Content.from_uri("https://media.gettyimages.com/id/1195994877/vector/democratic-donkey-and-republican-elephant-in-tv-debate.jpg?s=612x612&w=gi&k=20&c=1K-OwflyABXdG_xIbo_n7Ph3CRzI63vGx5G_sKmQz-Y=", media_type="image/jpg")
    ]
)

result = await agent.run(message)
print(result.text)

Why did the elephant and the donkey always get such good ratings?

Because their debates were never a "trunk" show! 🐘🎤🐴


## Multi-Turn Conversations

Agents are **stateless** and do not maintain any state internally between calls. To have a multi-turn conversation with an agent, you need to create an object to hold the conversation state and pass this object to the agent when running it.

### Creating a Thread

To create the conversation state object, call the `get_new_thread()` method on the agent instance:

In [13]:
thread = agent.get_new_thread()

### Running with a Thread

You can then pass this thread object to the `run` and `run_stream` methods on the agent instance, along with the user input. This maintains the conversation state between calls, allowing the agent to refer to previous messages:

In [14]:
# First turn
result1 = await agent.run("Tell me a joke about a pirate.", thread=thread)
print("Turn 1:")
print(result1.text)
print()

# Second turn - agent remembers the previous joke
result2 = await agent.run("Now add some emojis to the joke and tell it in the voice of a pirate's parrot.", thread=thread)
print("Turn 2:")
print(result2.text)

Turn 1:
Why did the pirate go to school?

Because he wanted to improve his "arrrrrrrticulation!"

Turn 2:
🦜 "Squawk! Why did the pirate go to school? 🎓 Arrr, it be because he wanted to improve his 'arrrrrrrticulation!' 🏴‍☠️ Arrr!" 🦜


### How Conversation History Works

> ⚠️ **Important:** The type of service used by the agent determines how conversation history is stored:
> 
> - **Chat Completion service** (like this example): Conversation history is stored in the `AgentThread` object and sent to the service on each call.
> - **Azure AI Agent service**: Conversation history is stored in the Azure AI Agent service and only a reference to the conversation is sent on each call.

### Multiple Independent Conversations

It's possible to have multiple, independent conversations with the same agent instance by creating multiple `AgentThread` objects. Since the agent doesn't maintain any state internally, these conversations will be fully independent:

In [15]:
# Create two separate conversation threads
thread1 = agent.get_new_thread()
thread2 = agent.get_new_thread()

# Start conversation 1 - about pirates
result1 = await agent.run("Tell me a joke about a pirate.", thread=thread1)
print("Thread 1 - Turn 1:")
print(result1.text)
print()

# Start conversation 2 - about robots (independent)
result2 = await agent.run("Tell me a joke about a robot.", thread=thread2)
print("Thread 2 - Turn 1:")
print(result2.text)
print()

# Continue conversation 1 - agent remembers the pirate joke
result3 = await agent.run("Now add some emojis to the joke and tell it in the voice of a pirate's parrot.", thread=thread1)
print("Thread 1 - Turn 2:")
print(result3.text)
print()

# Continue conversation 2 - agent remembers the robot joke
result4 = await agent.run("Now add some emojis to the joke and tell it in the voice of a robot.", thread=thread2)
print("Thread 2 - Turn 2:")
print(result4.text)

Thread 1 - Turn 1:
Why did the pirate go to school? 

Because he wanted to improve his “arrr ticulation”!

Thread 2 - Turn 1:
Why did the robot go on a diet?

Because it had too many bytes!

Thread 1 - Turn 2:
Polly wanna cracker! 🦜 Why did the pirate go to school? 🏴‍☠️ 

Because he wanted to improve his “arrr ticulation”! 😂💬 Arrr!

Thread 2 - Turn 2:
*Beep boop* 🤖 Why did the robot go on a diet? 🤔 

*Beep-beep* Because it had too many bytes! 🍔💻 *Whirrrrr* 


## Using Function Tools with an Agent

Function tools allow agents to call custom code when needed. You can turn any Python function into a function tool by passing it to the agent's `tools` parameter.

> ⚠️ **Important:** Not all agent types support function tools. Some might only support custom built-in tools. Agents created via chat clients (like this example) do support function tools.

### Creating a Simple Function Tool

Use Python's type annotations with `Annotated` and Pydantic's `Field` to provide descriptions that help the agent choose between different functions:

In [16]:
from typing import Annotated
from pydantic import Field

def get_weather(
    location: Annotated[str, Field(description="The location to get the weather for.")],
) -> str:
    """Get the weather for a given location."""
    return f"The weather in {location} is cloudy with a high of 15°C."

### Using the @tool Decorator

You can also use the `@tool` decorator to explicitly specify the function's name and description. If you don't specify them, the framework will automatically use the function's name and docstring as fallbacks:

In [17]:
from agent_framework import tool

@tool(name="weather_tool", description="Retrieves weather information for any location")
def get_weather_with_decorator(
    location: Annotated[str, Field(description="The location to get the weather for.")],
) -> str:
    return f"The weather in {location} is cloudy with a high of 15°C."

### Creating an Agent with Tools

When creating the agent, pass the function tool to the `tools` parameter:

In [18]:
# Create an agent with the weather tool
weather_agent = AzureOpenAIChatClient(credential=AzureCliCredential()).as_agent(
    instructions="You are a helpful assistant that can check the weather.",
    tools=get_weather
)

### Running the Agent with Tools

The agent will automatically call the `get_weather` function when needed:

In [19]:
# Ask about the weather - the agent will call our function tool
result = await weather_agent.run("What is the weather like in Amsterdam?")
print(result.text)

The weather in Amsterdam is currently cloudy, with a high temperature of 15°C.


### Class with Multiple Function Tools

You can create a class containing multiple function tools as methods. This is useful for organizing related functions together or when you want to share state between them:

In [20]:
class WeatherTools:
    def __init__(self):
        self.last_location = None

    def get_weather(
        self,
        location: Annotated[str, Field(description="The location to get the weather for.")],
    ) -> str:
        """Get the weather for a given location."""
        self.last_location = location
        return f"The weather in {location} is cloudy with a high of 15°C."

    def get_weather_details(self) -> str:
        """Get the detailed weather for the last requested location."""
        if self.last_location is None:
            return "No location specified yet."
        return f"The detailed weather in {self.last_location} is cloudy with a high of 15°C, low of 7°C, and 60% humidity."

### Using Class Methods as Tools

Create an instance of the class and pass its methods to the agent's `tools` parameter:

In [21]:
# Create instance and agent with multiple tools
weather_tools = WeatherTools()

multi_tool_agent = AzureOpenAIChatClient(credential=AzureCliCredential()).as_agent(
    instructions="You are a helpful weather assistant. You can get basic weather info and detailed weather info.",
    tools=[weather_tools.get_weather, weather_tools.get_weather_details]
)

# Test the multi-tool agent
result = await multi_tool_agent.run("What's the weather in Tokyo? Then give me the detailed forecast.")
print(result.text)

The current weather in Tokyo is cloudy, with a high of 15°C and a low of 7°C. The humidity is at 60%.


## Human-in-the-Loop: Function Tools Requiring Approval

When agents require user input (e.g., to approve a function call), this is called a **human-in-the-loop** pattern. An agent run that requires user input will complete with a response indicating what input is needed, instead of completing with a final answer.

The caller is responsible for getting the required input from the user and passing it back to the agent in a new run.

### Creating Tools with Approval Requirements

Use the `approval_mode="always_require"` parameter in the `@tool` decorator to indicate a function requires human approval:

In [22]:
# Standard tool - no approval needed
@tool
def get_basic_weather(location: Annotated[str, "The city and state, e.g. San Francisco, CA"]) -> str:
    """Get the current weather for a given location."""
    return f"The weather in {location} is cloudy with a high of 15°C."

# Tool that requires approval before execution
@tool(approval_mode="always_require")
def get_detailed_weather(location: Annotated[str, "The city and state, e.g. San Francisco, CA"]) -> str:
    """Get detailed weather information for a given location (requires approval)."""
    return f"The weather in {location} is cloudy with a high of 15°C, humidity 88%, wind 12 km/h NW."

### Creating the Agent with Approval-Required Tools

In [23]:
# Create an agent with both regular and approval-required tools
approval_agent = AzureOpenAIChatClient(credential=AzureCliCredential()).as_agent(
    instructions="You are a helpful weather assistant. Use get_detailed_weather for detailed forecasts.",
    tools=[get_basic_weather, get_detailed_weather]
)

### Checking for Approval Requests

When the agent needs to call an approval-required function, it returns a response with `user_input_requests` instead of executing the function directly:

In [24]:
# Ask for detailed weather - this will trigger an approval request
result = await approval_agent.run("What is the detailed weather like in Amsterdam?")

# Check if approval is needed
if result.user_input_requests:
    print("🔒 Approval Required!")
    for user_input_needed in result.user_input_requests:
        print(f"  Function: {user_input_needed.function_call.name}")
        print(f"  Arguments: {user_input_needed.function_call.arguments}")
else:
    print(result.text)

🔒 Approval Required!
  Function: get_detailed_weather
  Arguments: {"location":"Amsterdam, Netherlands"}


### Providing Approval and Continuing

Once you have the approval request, use `to_function_approval_response()` on the Content object - pass `True` to approve or `False` to reject. Then continue the conversation with the approval:

In [25]:
# If there was an approval request, provide approval and continue
if result.user_input_requests:
    user_input_needed = result.user_input_requests[0]
    
    # Simulate user approval (in a real app, this would be interactive)
    user_approval = True  # Set to False to reject
    print(f"✅ User approved: {user_approval}")
    
    # Create the approval response message using to_function_approval_response()
    approval_message = ChatMessage(
        role=Role.USER,
        contents=[user_input_needed.to_function_approval_response(user_approval)]
    )
    
    # Continue the conversation with the approval
    final_result = await approval_agent.run([
        "What is the detailed weather like in Amsterdam?",
        ChatMessage(role=Role.ASSISTANT, contents=[user_input_needed]),
        approval_message
    ])
    print(f"\n📊 Final Result:\n{final_result.text}")

✅ User approved: True

📊 Final Result:
The detailed weather in Amsterdam, Netherlands, is currently cloudy with a high of 15°C. The humidity is at 88%, and there is a northwest wind blowing at 12 km/h.


### Handling Multiple Approvals in a Loop

When working with multiple function calls that require approval, handle them in a loop until all are processed:

In [26]:
async def handle_approvals(query: str, agent) -> str:
    """Handle function call approvals in a loop."""
    current_input = query

    while True:
        result = await agent.run(current_input)

        if not result.user_input_requests:
            # No more approvals needed, return the final result
            return result.text

        # Build new input with all context
        new_inputs = [query]

        for user_input_needed in result.user_input_requests:
            print(f"🔒 Approval needed for: {user_input_needed.function_call.name}")
            print(f"   Arguments: {user_input_needed.function_call.arguments}")

            # Add the assistant message with the approval request
            new_inputs.append(ChatMessage(role=Role.ASSISTANT, contents=[user_input_needed]))

            # In practice, this would be interactive user input
            user_approval = True  # Replace with actual user input
            print(f"   ✅ Approved: {user_approval}")

            # Add the user's approval response using to_function_approval_response()
            new_inputs.append(
                ChatMessage(role=Role.USER, contents=[user_input_needed.to_function_approval_response(user_approval)])
            )

        # Continue with all the context
        current_input = new_inputs

# Test the approval handler
print("Testing approval handler with multiple locations:\n")
result_text = await handle_approvals("Get detailed weather for Seattle and Portland", approval_agent)
print(f"\n📊 Final Result:\n{result_text}")

Testing approval handler with multiple locations:

🔒 Approval needed for: get_detailed_weather
   Arguments: {"location": "Seattle, WA"}
   ✅ Approved: True
🔒 Approval needed for: get_detailed_weather
   Arguments: {"location": "Portland, OR"}
   ✅ Approved: True

📊 Final Result:
The detailed weather for both locations is as follows:

**Seattle, WA:**
- Condition: Cloudy
- High Temperature: 15°C
- Humidity: 88%
- Wind: 12 km/h NW

**Portland, OR:**
- Condition: Cloudy
- High Temperature: 15°C
- Humidity: 88%
- Wind: 12 km/h NW

The weather is quite similar in both cities.
